In [4]:
%load_ext autoreload
%autoreload 2

In [5]:
import os
os.chdir('..')

In [8]:
import os

import wandb
from omegaconf import OmegaConf

### Download locally the best models from wandb

---

In [4]:
CKPT_DIR = "/scratch/izar/cizinsky/rl-for-kinetics/best_models"

In [5]:
!mkdir -p {CKPT_DIR}

#### Get overview of the relevant runs

In [6]:
api = wandb.Api()

runs = api.runs("ludekcizinsky/rl-renaissance")

tagged_runs = [run for run in runs if "part2" in run.tags]

for run in tagged_runs:
    print(f"{run.id} | {run.name} | tags: {run.tags}")

kvxbygvv | zesty-bird-206 | tags: ['baseline', 'part2']


#### Download the best model for each run

In [7]:
run_ids = [run.id for run in tagged_runs]

for run_id in run_ids:
    print(f"Downloading checkpoint for run {run_id}...")
    run = next((run for run in runs if run.id == run_id), None)
    assert run is not None, "Run not found!"

    artifact_path = f"ludekcizinsky/rl-renaissance/{run.name}:v0"
    print(artifact_path)
    artifact = api.artifact(artifact_path, type="model")
    os.makedirs(f"{CKPT_DIR}/{run.name}", exist_ok=True)
    download_path = f"{CKPT_DIR}/{run.name}"

    # Config
    run_cfg = OmegaConf.create(run.config)
    OmegaConf.save(run_cfg, f"{download_path}/config.yaml")

    # Checkpoints
    artifact.download(download_path)
    print(f"Downloaded checkpoint to {download_path}.")

ludekcizinsky/rl-renaissance/zesty-bird-206:v0


wandb:   3 of 3 files downloaded.  


Downloaded checkpoint to /scratch/izar/cizinsky/rl-for-kinetics/best_models/zesty-bird-206.


### Inference

---

```bash
apptainer shell --nv --bind "$(pwd)":/home/renaissance/work --bind "/scratch/izar/$USER/rl-for-kinetics/output:/home/renaissance/output" --bind "/scratch/izar/$USER/rl-for-kinetics/best_models:/home/renaissance/best_models" /scratch/izar/$USER/images/renaissance_with_ml.sif
```

```bash
jupyter notebook --ip=0.0.0.0 --port=8888 --no-browser
````

In [1]:
!ls /home/renaissance/best_models

celestial-dragon-134  fragrant-plasma-133  sandy-blaze-130     vibrant-oath-142
chocolate-sponge-135  hearty-pine-137	   stilted-lake-138    zesty-bird-206
comfy-terrain-128     olive-forest-140	   sweet-eon-143
crisp-sunset-141      resilient-oath-127   upbeat-thunder-139


In [2]:
!ls /home/renaissance/best_models/zesty-bird-206

best_setup_e11_s3.pt  config.yaml  policy.pt  value.pt


In [23]:
import torch
from helpers.utils import setup_kinetic_env, get_incidence_rate
from helpers.ppo_agent import PolicyNetwork

In [9]:
selected_model = "zesty-bird-206"
ckpt_dir = f"/home/renaissance/best_models/{selected_model}"
best_setup_path = f"{ckpt_dir}/policy.pt"
config_path = f"{ckpt_dir}/config.yaml"

In [10]:
cfg = OmegaConf.load(config_path)

In [12]:
best_setup_filename = [f for f in os.listdir(ckpt_dir) if "best_setup" in f][0]
best_setup_path = f"{ckpt_dir}/{best_setup_filename}"
best_setup_path

'/home/renaissance/best_models/zesty-bird-206/best_setup_e11_s3.pt'

In [21]:
episode_str = best_setup_filename.split("_")[2]
episode = int(episode_str.split("e")[1])
step_str = best_setup_filename.split("_")[3].strip(".pt")
step = int(step_str.split("s")[1])
print(f"Best setup: episode {episode}, step {step}")

Best setup: episode 11, step 3


In [22]:
kinetic_env = setup_kinetic_env(cfg)
kinetic_env.logging_enabled = False

--------------------------------------------------
env:
  p_size: 384
  action_scale: 1
seed: 42
paths:
  names_km: data/varma_ecoli_shikki/parameter_names_km_fdp1.pkl
  output_dir: /home/renaissance/output
  met_model_name: varma_ecoli_shikki
device: cpu
logger:
  tags:
  - baseline
  entity: ludekcizinsky
  project: rl-renaissance
method:
  name: ppo_refinement
  actor_lr: 0.0003
  clip_eps: 0.2
  critic_lr: 0.001
  gae_lambda: 0.98
  parameter_dim: 384
  discount_factor: 0.99
  value_loss_weight: 0.5
reward:
  eig_partition: -2.5
training:
  batch_size: 25
  num_epochs: 10
  num_episodes: 100
  max_grad_norm: 0.5
  save_trained_models: true
  max_steps_per_episode: 50
  n_eval_samples_in_episode: 50
launch_cmd: train.py
constraints:
  max_km: 3
  min_km: -25
  ss_idx: 1712
lr_scheduler:
  name: constant

--------------------------------------------------
FYI: Loading kinetic and thermodynamic data.


In [11]:
device = "cpu"

In [24]:
best_setup = torch.load(best_setup_path)

In [ ]:
max_steps_per_episode = 10 # cfg.training.max_steps_per_episode
num_samples = 50
for i in range(10):
    incidence_rate, all_max_eigs = get_incidence_rate(kinetic_env, best_setup, num_samples, device)
    print(f"[Eval {i}] Incidence rate: {incidence_rate}")